In [38]:
import bs4
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup 
from collections import defaultdict

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

In [39]:
'''
The data required for crawling is saved in a dynamic table that updates with more rows everytime the page is
scrolled to the bottom, this script will scroll to the bottom of the page revealing all data.
'''

def scrollPage(URL):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(URL_test)
    time.sleep(10)                       # Wait for page to load

    last_height = driver.execute_script('return document.querySelector("table").scrollHeight')
    element = driver.find_element(By.TAG_NAME, "table")

    while True:
        element.send_keys(Keys.END)      # Scroll to end of available page
        time.sleep(3)                    # Wait for new content to load

        new_height = driver.execute_script('return document.querySelector("table").scrollHeight')
        if new_height == last_height:    # End of page
             break
        last_height = new_height

    return driver.page_source            # html

In [40]:
'''
After all the table is loaded, this script will crawl the page and retrieve the data.
'''

def crawlHTML(htmlText):    
    soup = BeautifulSoup(htmlText)
    titles, columns = {}, defaultdict(list)
    i = 0
    
    # Save the collumn number of each header title for later access to its values
    for t in soup.findAll("th"):         
        titles[t.find("span").string.strip()] = i
        i += 1
    
    # Create a dictionary of {header title : list of collumn values}
    for title in titles:
        for value in soup.findAll("td", attrs={"data-col":str(titles[title])}):
            columns[title].append(value.text)
    
    return pd.DataFrame(columns)         # DataFrame

In [41]:
def saveDataFrameToCSV(df, path):
    df.to_csv(path, index=False)

In [42]:
'''
Driver Code
'''

# URL:       Page with all wildfire records since 2014, USA only
# URL_test:  Page with all ongoing wildfires, USA only, short table

URL = "https://data-nifc.opendata.arcgis.com/datasets/wfigs-wildland-fire-locations-full-history/explore?location=0.000000%2C0.000000%2C0.00&showTable=true"
URL_test = "https://data-nifc.opendata.arcgis.com/datasets/wfigs-current-wildland-fire-locations/explore?location=-0.000000%2C0.000000%2C7.50&showTable=true"
path_csv = "Wildland Fires.csv"

htmlText = scrollPage(URL_test)
df = crawlHTML(htmlText)
saveDataFrameToCSV(df, path_csv)

print("Task complete")



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\tspie\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


Task complete


In [43]:
df.head()

,ABCD Misc,ADS Permission State,Calculated Acres,Containment Date Time,Control Date Time,Daily Acres,Discovery Acres,Dispatch Center ID,Estimated Cost To Date,Final Fire Report Approved Date,...,Unique Fire Identifier,WFDSS Decision Status,Created By System,Modified By System,Is Dispatch Complete,Organizational Assessment,Strategic Decision Publish Date,Created On Date Time,Modified On Date Time,Source
0,,DEFAULT,,,,324,1,KYKIC,,,...,2021-KYDBF-210099,No Decision,wildcad,wildcad,0,,,"1/23/2021, 6:11 PM","12/9/2021, 6:22 PM",IRWIN
1,,DEFAULT,,,,18,1,SDGPC,,,...,2021-SDSDS-210107,No Decision,wildcad,wildcad,0,,,"2/17/2021, 8:57 PM","12/10/2021, 9:10 PM",IRWIN
2,,DEFAULT,,,,10,1,SDGPC,,,...,2021-SDSDS-210052,No Decision,wildcad,wildcad,0,,,"2/17/2021, 8:57 PM","12/10/2021, 9:09 PM",IRWIN
3,,DEFAULT,,,,156,1,UTRFC,,,...,2021-UTFIF-210015,No Decision,wildcad,wildcad,0,,,"2/9/2021, 12:40 AM","12/9/2021, 9:56 PM",IRWIN
4,,DEFAULT,,,,19.25,1,COPBC,,,...,2021-CORGF-000036,No Decision,wildcad,wildcad,0,,,"1/19/2021, 10:46 PM","12/9/2021, 10:27 PM",IRWIN
